# MeLi Challenge

Build a Machine Learning model to predict next purchase based on the user’s navigation history. 

Two Data sets are provided. 

- One of them has the user history information along with the target variable (item bought)
- The other one has the metadata for all the items in MeLi's inventory, which can be used to obtain information about the items the usar has seen, searched or bought.

En base a los archivos Json previamente decoded en un pandas data frame, construyo un data set con informacion basada en el item mas visto por el usuario.

Esta notebook trata la informacion del json como string, lo que conlleva una mala performance en computational complexity. En notebooks posteriores y actualmente en desarrollo trabajo con herramientas mas eficientes como el 
json_normalize con record_path para leer el json a un df.

Esta Notebook tiene una continuacion en MeLiChallengeDataSetAnalysis

In [3]:
import pandas as pd
import datetime
import re

In [4]:
trainpath = 'C:\\Users\\desar\\OneDrive\\Escritorio\\data_science\\MeLi Challenge\\train_set.csv'
testpath = 'C:\\Users\\desar\\OneDrive\\Escritorio\\data_science\\MeLi Challenge\\test_set.csv'
train = pd.read_csv(trainpath)
test = pd.read_csv(testpath)

In [5]:
train.head()

,user_history,item_bought
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207


In [6]:
#como el train set es muy pesado, creo un df de prueba mas chico para meter mano y cuando vea que
#es lo que quiero ok entonces recien lo hago en el otro

df = train.iloc[:100]
df.shape

(100, 2)

In [7]:
df.head()

,user_history,item_bought
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207


In [8]:
def most_viewed(x):
    """obtiene el articulo most viewed"""
    try:
        # Creo un df con los items del findAll
        df1 = pd.DataFrame({'Item': re.findall(r'\s\d+,', x)}) #la coma me asegura q el nro que encuentra es del item
  
        #ahora creo otro df con el value counts de cada valor de la lista   
        df2 = pd.DataFrame(data=df1['Item'].value_counts()) 
  
        # Ahora hago que me devuelva el primer index de ese df2
        return df2.index[0] #que puede o no ser el max exclusivo
    
    except IndexError:
        
        return None

In [9]:
df['most_viewed'] = df['user_history'].apply(most_viewed)

<ipython-input-9-7bd0a173c4c0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['most_viewed'] = df['user_history'].apply(most_viewed)


In [10]:
def cantidad_vistas(x):
    """obtiene la cantidad de veces que el user vio el articulo most viewed"""
    try:
        # Creo un df con los items del findAll
        df1 = pd.DataFrame({'Item': re.findall(r'\s\d+,', x)})
  
        #ahora creo otro df con el value counts de cada valor de la lista   
        df2 = pd.DataFrame(data=df1['Item'].value_counts()) 
  
        # Ahora hago que me devuelva el primer valor de ese df2
        return int(df2.iloc[0,0]) #que puede o no ser el max exclusivo
    
    except IndexError:
        
        return None

In [11]:
df['veces_visto'] = df['user_history'].apply(cantidad_vistas)

<ipython-input-11-63a3e5430863>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['veces_visto'] = df['user_history'].apply(cantidad_vistas)


In [12]:
df['most_viewed'].isna().value_counts()

False    93
True      7
Name: most_viewed, dtype: int64

In [13]:
train['most_viewed'] = train['user_history'].apply(most_viewed)
test['most_viewed'] = test['user_history'].apply(most_viewed)

In [14]:
train['m_view_veces'] = train['user_history'].apply(cantidad_vistas)
test['m_view_veces'] = test['user_history'].apply(cantidad_vistas)

In [17]:
train.shape

(413163, 4)

In [18]:
def limpiar_items(x):
    """Limpio las comas de la most viewed"""
    if x == None:
        return None
    else:
        return str(x)[:-1] 

train['most_view'] = train['most_viewed'].apply(limpiar_items)
test['most_view'] = test['most_viewed'].apply(limpiar_items)

train.drop('most_viewed', axis = 1, inplace = True)
test.drop('most_viewed', axis = 1, inplace = True)

In [19]:
train.head()

,user_history,item_bought,m_view_veces,most_view
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,16.0,1615991
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737,3.0,228737
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110,2.0,248595
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370,5.0,505541
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207,15.0,1313192


In [20]:
def most_viewed_2(x):
    """obtiene el articulo most viewed"""
    try:
        df1 = pd.DataFrame({'Item': re.findall(r'\s\d+,', x)})  
        df2 = pd.DataFrame(data=df1['Item'].value_counts()) 
        return df2.index[1] #que puede o no ser el max exclusivo
    except IndexError:
        return None
    
def most_viewed_3(x):
    """obtiene el articulo most viewed"""
    try:
        df1 = pd.DataFrame({'Item': re.findall(r'\s\d+,', x)})  
        df2 = pd.DataFrame(data=df1['Item'].value_counts()) 
        return df2.index[2] #que puede o no ser el max exclusivo
    except IndexError:
        return None

In [21]:
for data in [train, test]:
    data['most view_2'] = data['user_history'].apply(most_viewed_2)
    data['most view_3'] = data['user_history'].apply(most_viewed_3)
    data['most_view_2'] = data['most view_2'].apply(limpiar_items)
    data['most_view_3'] = data['most view_3'].apply(limpiar_items)
    data.drop(['most view_2', 'most view_3'], axis = 1, inplace = True)

In [22]:
train.head()

,user_history,item_bought,m_view_veces,most_view,most_view_2,most_view_3
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,16.0,1615991,1786148,None
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737,3.0,228737,1943604,206667
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110,2.0,248595,None,None
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370,5.0,505541,1230082,937557
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207,15.0,1313192,1447583,2049207


In [23]:
#fillea con ceros los none de most views

In [102]:
#Lo cambio a 500 asi es un poco mas significativo

df = train.iloc[:500]

In [27]:
#sacar cant de articulos vistos
def articulos_vistos(x):
    """obtiene el total de articulos que el user vio previo al a compra"""
    try:
        # Creo un df con los items del findAll
        df1 = pd.DataFrame({'Item': re.findall(r'\s\d+,', x)})
  
        #ahora creo otro df con el value counts de cada valor de la lista   
        df2 = pd.DataFrame(data=df1['Item'].value_counts()) 
  
        # Ahora hago que me el total de articulos vistos (la len del df)
        return len(df2) 
    except IndexError:
        return None

In [31]:
for data in [train, test]:
    data['art_vistos'] = data['user_history'].apply(articulos_vistos)

In [103]:
df.head(5)

,item_bought,m_view_veces,most_view,most_view_2,most_view_3,art_vistos,time_buy,view_entries,search_entries
0,1748830,16.0,1615991,1786148,None,2,1 days 08:02:59,18,1
1,228737,3.0,228737,1943604,206667,6,1 days 01:43:25,8,5
2,1909110,2.0,248595,None,None,1,0 days 00:35:47,2,0
3,1197370,5.0,505541,1230082,937557,3,3 days 17:39:00,7,6
4,2049207,15.0,1313192,1447583,2049207,17,5 days 06:14:20,46,23


In [76]:
def time_delta(x):
    """Devuelve el rango de tiempo que se tomo el usuario para la compra"""
    df1 = pd.DataFrame({'Item': re.findall(r'\d\d\d\d-\d\d-\d\dT\d\d:\d\d:\d\d.', x)})
    df2 = pd.DataFrame(data = {df1['Item'].value_counts()}) 
    df2['Item'] = pd.to_datetime(df2['Item'])
    df2.sort_values()
    
    return df2.iloc[-1] - df2.iloc[0]

for data in [train, test]:
    data['time_buy'] = data['user_history'].apply(time_delta)
    
train.head(10)    

,user_history,item_bought,m_view_veces,most_view,most_view_2,most_view_3,art_vistos,time_buy
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,16.0,1615991,1786148,None,2,1 days 08:02:59
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737,3.0,228737,1943604,206667,6,1 days 01:43:25
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110,2.0,248595,None,None,1,0 days 00:35:47
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370,5.0,505541,1230082,937557,3,3 days 17:39:00
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207,15.0,1313192,1447583,2049207,17,5 days 06:14:20
5,"[{'event_info': 'TAMPA TRASEIRA IPHONE8', 'eve...",1046119,2.0,1135701,2029385,794504,3,1 days 20:52:47
6,"[{'event_info': 400631, 'event_timestamp': '20...",1206282,1.0,1998468,400631,1156202,3,3 days 08:11:52
7,"[{'event_info': 'SECADOR TAIFF', 'event_timest...",1393318,1.0,375282,None,None,1,0 days 00:00:42
8,"[{'event_info': 'AUDIFONOS BASS', 'event_times...",2056361,1.0,1894398,1591096,None,2,0 days 00:05:32
9,"[{'event_info': 1142550, 'event_timestamp': '2...",1916890,2.0,1142550,426711,29678,4,0 days 00:09:22


In [98]:
def view_entries(x):
    '''Devuelve la cantidad de entries de categoria view'''
    views = re.findall(r"'view'", x)
    return len(views)

def search_entries(x):
    '''Devuelve la cantidad de entries de categoria search'''
    searchs = re.findall(r"'search'", x)
    return len(searchs)

for data in [train,test]:
    data['view_entries'] = data['user_history'].apply(view_entries)
    data['search_entries'] = data['user_history'].apply(search_entries)
    
train.head(10)

,user_history,item_bought,m_view_veces,most_view,most_view_2,most_view_3,art_vistos,time_buy,view_entries,search_entries
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,16.0,1615991,1786148,None,2,1 days 08:02:59,18,1
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737,3.0,228737,1943604,206667,6,1 days 01:43:25,8,5
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110,2.0,248595,None,None,1,0 days 00:35:47,2,0
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370,5.0,505541,1230082,937557,3,3 days 17:39:00,7,6
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207,15.0,1313192,1447583,2049207,17,5 days 06:14:20,46,23
5,"[{'event_info': 'TAMPA TRASEIRA IPHONE8', 'eve...",1046119,2.0,1135701,2029385,794504,3,1 days 20:52:47,4,4
6,"[{'event_info': 400631, 'event_timestamp': '20...",1206282,1.0,1998468,400631,1156202,3,3 days 08:11:52,3,12
7,"[{'event_info': 'SECADOR TAIFF', 'event_timest...",1393318,1.0,375282,None,None,1,0 days 00:00:42,1,2
8,"[{'event_info': 'AUDIFONOS BASS', 'event_times...",2056361,1.0,1894398,1591096,None,2,0 days 00:05:32,2,11
9,"[{'event_info': 1142550, 'event_timestamp': '2...",1916890,2.0,1142550,426711,29678,4,0 days 00:09:22,5,1


In [99]:
#hasta aca llego con los datos del user history, seguro se puede hacer muchisimo mas
#pero hasta aca dan los conocimientos
#exporto el train set y el test set ya sin el apartado de user history

for d in [train,test]:
    d.drop('user_history', axis = 1, inplace = True)

In [100]:
train.head()

,item_bought,m_view_veces,most_view,most_view_2,most_view_3,art_vistos,time_buy,view_entries,search_entries
0,1748830,16.0,1615991,1786148,None,2,1 days 08:02:59,18,1
1,228737,3.0,228737,1943604,206667,6,1 days 01:43:25,8,5
2,1909110,2.0,248595,None,None,1,0 days 00:35:47,2,0
3,1197370,5.0,505541,1230082,937557,3,3 days 17:39:00,7,6
4,2049207,15.0,1313192,1447583,2049207,17,5 days 06:14:20,46,23


In [101]:
train.to_csv('train_set.csv', index = False)
test.to_csv('test_set.csv', index = False)